# Note
Please note that this code is NOT optimised and initialy there is no attempt to optimise. 
There is no attempt to fine tune models at this stage 
The goal is to get a test multiple hypothesis and see what best concepts can be build in. 

# Result Summary:
Accuracy achieved 97% on 8 possible targets
Accuracy achieved : 84% on 315 possible targets 

# TO-DO
1) compute words which have 70% similar characters

2) create a number based represenation of the words in the target. this will help the random forest to distinguish the target names

3) use the represenation to come up with different metrics for the words with similar characters

4) Build the dictionary for key probability -- pending

5) build the similarity / probability score for each word match or not -- pending

6) use the number represenation to add uniqueness to each word  -- pending

In [31]:
cd D:\GG_Clean

D:\GG_Clean


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import collections 
from collections import Counter 
import string

from random import shuffle # this helps in shuffling numbers randomly 

In [33]:
data = pd.read_excel("file_for_github.xlsx") # file is available on the github page.... incase you wish to run on your own

In [34]:
data.head(5)

,input_name,final_name
0,COMPUTERSHARE TRUST COMPANY OF CANADA,COMPUTERSHARE TRUST COMPANY
1,"""COMPUTERSHARE TRUST COMPANY OF CANADA"" (THE ""...",COMPUTERSHARE TRUST COMPANY
2,* NEW PARTY COMPUTERSHARE TRUST COMPANY OF CANADA,COMPUTERSHARE TRUST COMPANY
3,* NEW PARTY COMPUTERSHARE TRUST COMPANY OF CAN...,COMPUTERSHARE TRUST COMPANY
4,*COMPUTERSHARE TRUST COMPANY OF CANADA,COMPUTERSHARE TRUST COMPANY


here the user will provide a train data-set with pre-cleaned data from input_name to final_name. 

once the model is trained it can be used to clean other similar such stuctures. 

In this data set there are 8 uniuque elements in  final_name. this is our target variable.

The input_name is the raw input. it can vary dramaticly in number of characters, number of words, the very charectors themselves. 

# Create features

## to do 
Create features that meet the following requirements 

1) Words with complete match 

2) words with 70% match

3) Words with complete match but weighted based on a number for the target name

4) Words with 70% match but weighted based on a number for the target name

In [ ]:
# create , fill , and read a dataframe for each row to get values for 
exceptions_captured = []
for main_index in data.index:
    try:
        cut_off_percentage = 0.7
        word_100 = 0
        word_75 = 0

        df_word_similarity = pd.DataFrame(columns=[ str(i) for i in data.loc[main_index , 'input_name'].split()]
                                          , index= [str(j) for j in data.loc[main_index , 'final_name'].split()])

        # fill in the dataframe 
        for i in data.loc[main_index , 'input_name'].split():
            counter_i = Counter(i)

            for j in data.loc[main_index , 'final_name'].split():
                counter_j = Counter(j)
                diff_in_counters = counter_i & counter_j

                overlap_i_j = float(sum(diff_in_counters.values()) / sum(counter_i.values()))
                df_word_similarity.loc[i,j] = overlap_i_j

        # read from the dataframe to get the word_100 and word_75

        for i in df_word_similarity.index:
            y = pd.Series(df_word_similarity.loc[str(i) , :]).astype('float64') # the float is done so that we can get y.idxmax()
            if y.max() == 1 :
                word_100 += 1
            elif y.max() >= cut_off_percentage:
                word_75 += 1
            # for later when we need the name of the word we can do y.idxmax() to get the name of the column.. 
            #########################
            # here we need to add the weights of the target word to the data 
            # this will help to get the model to focus on the target word
            #########################

        # update the main dataframe 

        data.loc[main_index , 'word_100'] = word_100
        data.loc[main_index , 'word_75']  = word_75  # later we can change word_75 to word_cut_off
    except KeyboardInterrupt:
        raise

    except Exception as e:
        exceptions_captured.append(main_index)
        pass






# generate new features based on  number calculation 


Get 2 new features from the earlier 2 features by normalisation and multiplication by the number for each target group o

In [ ]:
# Generate the  number for each final_name ... for example Td Bank = 300

unique_final_names = list(data.final_name.unique())
shuffle(unique_final_names) # here inplace is true by default 
                          
dict_final_word_prime_number = {}

for i , j in zip(unique_final_names ,range(1, 100000000000 , 300 )) :
    
    dict_final_word_prime_number[i] = j


In [ ]:
# create 2 new features - based on normalisation 

data['word_100_normalised'] = ""
data["word_75_normalised"] = ""

for i in data.index:
    len_input_name = len(data.loc[i , 'input_name'].split())
    len_input_name = [len_input_name if len_input_name != 0 else 1]
    
    data.loc[i, 'word_100_normalised'] = data.loc[i,'word_100']/len_input_name
    data.loc[i, 'word_75_normalised'] = data.loc[i,'word_75']/len_input_name
    
    

In [ ]:
# generate 2 new features with the weightage of  numbers for each Fianl name .... This will help to individualize the 
# data in each final_name 

data['word_100_prime'] = ""
data["word_75_prime"] = ""
data['word_100_normalised_prime'] = ""
data["word_75_normalised_prime"] = ""

for i in data.index:
   
    y = dict_final_word_prime_number.get(data.loc[i, 'final_name'])
    
    data.loc[i, 'word_100_prime'] = data.loc[i,'word_100']*y
    data.loc[i, 'word_75_prime'] = data.loc[i,'word_75']*y
    data.loc[i, 'word_100_normalised_prime'] = data.loc[i,'word_100_normalised']*y
    data.loc[i, 'word_75_normalised_prime'] = data.loc[i,'word_75_normalised']*y
    
data.fillna(value= 0 , inplace= True)
    

# Create Model 

Here i am using a Random Forest. There is no attempt to fine-tune. 

In [ ]:
# load dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve

def metrics(y_test , y_pred):
    print('accuracy_score is {}'.format(accuracy_score(y_test , y_pred)))
    print('\n confusion_matrix is \n {}'.format(confusion_matrix(y_test,y_pred)))
    print('\n classification report is \n {}'.format(classification_report(y_test, y_pred)))

In [ ]:
#  model development 

features= [
 'word_100',
 'word_75',
 'word_100_normalised',
 'word_75_normalised',
 'word_100_prime',
 'word_75_prime',
 'word_100_normalised_prime',
 'word_75_normalised_prime'
    ]

target = ['final_name']

X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.33, random_state=42)


In [ ]:
# generate an instance of the model 

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

preds = rf.predict(X_test)




# Check Metrics

In [ ]:
metrics(y_test ,preds)

accuracy_score is 0.9632352941176471

 confusion_matrix is 
 [[151   2   0   0   0   0   0   0]
 [  2 283   0   0   0   0   0   0]
 [  0   1  14   1   0   0   0   2]
 [  0   0   2   3   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  0   0   0   0   0 197   0   0]
 [  0   0   0   0   0   0   2   0]
 [  0  11   3   0   0   0   0   5]]

 classification report is 
                                     precision    recall  f1-score   support

       COMPUTERSHARE TRUST COMPANY       0.98      0.99      0.98       153
                           TD BANK       0.95      0.99      0.97       285
                   TD CANADA TRUST       0.74      0.78      0.76        18
   TD FINANCIAL SERVICES HOME INC.       0.75      0.60      0.67         5
TD HOME AND AUTO INSURANCE COMPANY       0.00      0.00      0.00         1
                  TD TRUST COMPANY       1.00      1.00      1.00       197
                     TD WATERHOUSE       1.00      1.00      1.00         2
             TORONTO

In [ ]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',  ascending=False)

In [ ]:
feature_importances

,importance
word_100,0.257767
word_100_normalised_prime,0.226924
word_100_prime,0.221265
word_75_normalised_prime,0.088959
word_75_prime,0.088607
word_100_normalised,0.058772
word_75_normalised,0.047631
word_75,0.010075


# Visualize 

In [ ]:
# dependencies 

import matplotlib.pyplot as plt
import numpy as np
import PIL
import pydotplus
import warnings
from glob import glob
from IPython.display import display, Image
# from sklearn.datasets import load_iris
# from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
%matplotlib inline
warnings.filterwarnings("ignore")

In [ ]:
# source credit: https://www.youtube.com/watch?v=SMGEmCOUBUw&t=81s 

def save_decision_trees_as_png(clf, iteration, feature_name, target_name):
    file_name = "tree" + str(iteration) + ".png"
    dot_data = export_graphviz(
        clf,
        out_file=None,
        feature_names=feature_name,
        class_names=target_name,
        rounded=True,
        proportion=False,
        precision=2,
        filled=True,
        )
    graph = pydotplus.graph_from_dot_data(dot_data)
    graph.write_png(file_name)
    print("Decision Tree {} saved as png file".format(iteration + 1))

In [ ]:
feature_names = features
target_names = data['final_name'].unique()

for i in range(len(rf.estimators_)):
    save_decision_trees_as_png(rf.estimators_[i], i, feature_names, target_names)

Decision Tree 1 saved as png file
Decision Tree 2 saved as png file
Decision Tree 3 saved as png file
Decision Tree 4 saved as png file


In [ ]:
images = [ PIL.Image.open(f) for f in glob('./*.png') ]

for im in images:
    display(Image(filename=im.filename, retina=True))
    im.close()

please click on any tree to expand the view